In [12]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.3 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

food = load_dataset("food101",split="train[:5000]")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00008.parquet:   0%|          | 0.00/490M [00:00<?, ?B/s]

train-00001-of-00008.parquet:   0%|          | 0.00/464M [00:00<?, ?B/s]

train-00002-of-00008.parquet:   0%|          | 0.00/472M [00:00<?, ?B/s]

...

validation-00000-of-00003.parquet:   0%|          | 0.00/423M [00:00<?, ?B/s]

validation-00001-of-00003.parquet:   0%|          | 0.00/413M [00:00<?, ?B/s]

validation-00002-of-00003.parquet:   0%|          | 0.00/426M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/75750 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/25250 [00:00<?, ? examples/s]

In [3]:
food_split = food.train_test_split(0.2)

In [4]:
labels = food_split['train'].features['label'].names
label2id, id2label = {}, {}
for i , label in enumerate(labels):
    label2id[label] = i,
    id2label[i] = label

In [13]:
len(label2id)

101

In [36]:
from transformers import AutoImageProcessor, AutoModelForImageClassification, TrainingArguments, Trainer

processor = AutoImageProcessor.from_pretrained("facebook/convnext-base-224")

preprocessor_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

In [37]:
processor.size

{'shortest_edge': 224}

In [38]:
size = processor.size

In [27]:
size

{'shortest_edge': 224}

In [39]:
from torchvision.transforms import RandomResizedCrop ,Compose ,Resize ,Normalize ,ToTensor

In [40]:
transform = Compose([
    RandomResizedCrop(size['shortest_edge']),
    ToTensor(),
    Normalize(mean=processor.image_mean,std=processor.image_std)
])

In [41]:
from PIL import Image

def process_image(samples):
    samples['pixel_values'] = [transform(img.convert('RGB')) for img in samples['image']]
    del samples['image']
    return samples

In [42]:
food_split_proc = food_split.with_transform(process_image)

In [43]:
from transformers import AutoModelForImageClassification ,TrainingArguments ,Trainer,default_data_collator
import evaluate
import numpy as np
data_collator = default_data_collator
metric = evaluate.load('accuracy')

In [44]:
def compute_metrics(predictions_and_labels):
    predictions,labels = predictions_and_labels
    predictions = np.argmax(predictions,axis=-1)
    return metric.compute(predictions=predictions,references=labels)

In [45]:
model = AutoModelForImageClassification.from_pretrained('facebook/convnext-base-224',num_labels=len(labels),id2label=id2label,label2id=label2id,ignore_mismatched_sizes=True)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/354M [00:00<?, ?B/s]

Some weights of ConvNextForImageClassification were not initialized from the model checkpoint at facebook/convnext-base-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1024]) in the checkpoint and torch.Size([101, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([101]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
training_args = TrainingArguments(
    # output_dir="my_model",
    output_dir="./results",
    report_to=[],
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="epoch",
    weight_decay=0.01,
    remove_unused_columns=False,
    save_strategy="epoch",
    learning_rate=0.001,
    num_train_epochs=7,
    logging_steps=10,
    push_to_hub=False,
    metric_for_best_model="accuracy"
)

In [47]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    train_dataset=food_split_proc['train'],
    eval_dataset=food_split_proc['test'],
    processing_class=processor
)

In [34]:
import os
os.environ["WANDB_DISABLED"] = "true"
import torch
print("CUDA available?", torch.cuda.is_available())
print("Device being used:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")



CUDA available? True
Device being used: Tesla T4


In [48]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.726700,0.720753,0.759000
2,0.554600,0.640576,0.796000
3,0.340300,0.573640,0.827000
4,0.178100,0.532086,0.829000
5,0.188000,0.516233,0.845000
6,0.100500,0.461663,0.868000
7,0.084200,0.402999,0.900000


TrainOutput(global_step=1750, training_loss=0.3774721326998302, metrics={'train_runtime': 1615.1447, 'train_samples_per_second': 17.336, 'train_steps_per_second': 1.083, 'total_flos': 2.217060401504256e+18, 'train_loss': 0.3774721326998302, 'epoch': 7.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/model_CV/facebook")